In [31]:
FILE_PATH = 'day_2_data.txt'

# Problem 1

In [44]:
keymap_dict = {
    'red': 12,
    'green': 13,
    'blue': 14,
}

answer = 0
with open(FILE_PATH, mode='r') as file:
    for line in file:
        possible = True
        line = line.strip()
        id_, line = line.split(':')
        for balls in line.split(';'):
            for ball in balls.split(','):
                n, color = ball.split()
                if int(n) > keymap_dict.get(color, 0):
                    possible = False
        if possible:
            answer += int(id_.split()[-1])
print(answer)

2795


# Problem 2

In [48]:
from collections import defaultdict

In [64]:
answer = 0
with open(FILE_PATH, mode='r') as file:
    for line in file:
        possible = True
        line = line.strip()
        id_, line = line.split(':')
        value_dict = defaultdict(int)
        for balls in line.split(';'):
            for ball in balls.split(','):
                n, color = ball.split()
                n = int(n)
                value_dict[color] = max(value_dict[color], n)
        score = 1
        for value in value_dict.values():
            score *= value
        answer += score
print(answer)

75561
